In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
train = gpd.read_file("../../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp")
train.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry
0,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield,POINT (144.95230 -37.78119)
1,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield,POINT (144.93932 -37.78814)
2,19972,Macaulay Railway Station (North Melbourne),-37.794267,144.936166,1,Upfield,POINT (144.93617 -37.79427)
3,19973,North Melbourne Railway Station (West Melbourne),-37.807419,144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.94257 -37.80742)
4,19974,Clifton Hill Railway Station (Clifton Hill),-37.788657,144.995417,1,"Mernda,Hurstbridge",POINT (144.99542 -37.78866)


In [4]:
train.to_csv('../../data/raw/train_station.csv')

In [5]:
# data about the location of each train station
train = pd.read_csv("../../data/raw/train_station.csv")   

In [6]:
# passenger flow volume data for each station
pas = pd.read_csv("../../data/raw/Annual metropolitan train station entries 2020-21.csv") 

In [7]:
pas["STOP_ID"] = pas["Stop_ID"]  # create merging attribute

In [8]:
print(pas.isnull().sum())

Fin_year               0
Stop_ID                0
Stop_name              0
Stop_lat               0
Stop_long              0
Pax_annual             0
Pax_weekday            0
Pax_norm_weekday       0
Pax_sch_hol_weekday    0
Pax_Saturday           0
Pax_Sunday             0
Pax_pre_AM_peak        0
Pax_AM_peak            0
Pax_interpeak          0
Pax_PM_peak            0
Pax_PM_late            0
STOP_ID                0
dtype: int64


### Merge the location of train station and passenger flow volume 

In [9]:
# merge data on "STOP_ID"
merge = pd.merge(train, pas, on="STOP_ID", how="outer") 

In [10]:
merge.head()

,Unnamed: 0,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry,Fin_year,Stop_ID,...,Pax_weekday,Pax_norm_weekday,Pax_sch_hol_weekday,Pax_Saturday,Pax_Sunday,Pax_pre_AM_peak,Pax_AM_peak,Pax_interpeak,Pax_PM_peak,Pax_PM_late
0,0.0,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield,POINT (144.95230116 -37.7811929699999),FY20-21,19970,...,350,350,500,250,200,50,50,100,150,50
1,1.0,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield,POINT (144.93932316 -37.78813998),FY20-21,19971,...,300,300,300,200,150,50,50,100,100,50
2,2.0,19972,Macaulay Railway Station (North Melbourne),-37.794267,144.936166,1,Upfield,POINT (144.93616596 -37.794267),FY20-21,19972,...,350,350,300,200,150,50,50,100,150,50
3,3.0,19973,North Melbourne Railway Station (West Melbourne),-37.807419,144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.94257 -37.80741897),FY20-21,19973,...,1950,2000,1650,1950,1300,50,350,500,950,150
4,4.0,19974,Clifton Hill Railway Station (Clifton Hill),-37.788657,144.995417,1,"Mernda,Hurstbridge",POINT (144.99541692 -37.78865703),FY20-21,19974,...,1000,1050,950,900,700,50,350,250,300,100


In [11]:
merge["Pax_annual"].describe()  # check the data validity

count    2.220000e+02
mean     3.505405e+05
std      7.405339e+05
min      1.850000e+03
25%      1.096250e+05
50%      1.978000e+05
75%      3.395500e+05
max      8.528000e+06
Name: Pax_annual, dtype: float64

### Select useful attributes

In [12]:
# determine useful attributes
merge=merge[["STOP_ID","STOP_NAME","LATITUDE",
             "LONGITUDE","ROUTEUSSP","geometry","Fin_year","Pax_annual"]] 

In [13]:
print(merge.isnull().sum())

STOP_ID       0
STOP_NAME     2
LATITUDE      2
LONGITUDE     2
ROUTEUSSP     2
geometry      2
Fin_year      0
Pax_annual    0
dtype: int64


In [14]:
merge=merge.dropna() #delete missing value

In [15]:
merge.to_csv('../../data/curated/traffic_dataset.csv')